In [63]:
#import
import math
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
import xgboost as xgb
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import torch.nn.functional as F
import matplotlib.pyplot as plt

from random import randint
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [64]:
#make screen wide
from IPython.core.display import display, HTML
from IPython import get_ipython as get_ipython
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important;margin-left:-30px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

# FCNN

In [ ]:
Loss function
lr
optimizer
architecture
batch_size

In [65]:
class InterpolDataSet(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        self.len = len(labels)

    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        return self.len

class InterpolDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.df = None
    
    def setup(self, stage):
        # data reading
        df = pd.read_csv('./data/clasdb_pi_plus_n.txt', delimiter='\t', header=None)
        df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']

        df.loc[8314:65671, 'Ebeam'] = 5.754
        df['phi'] = df.phi.apply(lambda x: math.radians(x))
        df = df.drop('id', axis=1)
        df = df.iloc[df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']].drop_duplicates().index]
        self.df = df
        
        #train test split
        feature_data = df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']]
        label_data = df['dsigma_dOmega']
        
        train_feature_data, test_feature_data, train_label_data, test_label_data = train_test_split(feature_data, label_data, 
                                                                                                    test_size=0.1, random_state=42)
        
        
        self.train_dataset = InterpolDataSet(torch.tensor(train_feature_data.values, dtype=torch.float32), 
                                             torch.tensor(train_label_data.values, dtype=torch.float32))
        self.test_dataset = InterpolDataSet(torch.tensor(test_feature_data.values, dtype=torch.float32), 
                                            torch.tensor(test_label_data.values, dtype=torch.float32))

    def train_dataloader(self):
        return DataLoader(dataset = self.train_dataset, batch_size = 8, shuffle = False, num_workers=0)

    def val_dataloader(self):
        return DataLoader(dataset = self.test_dataset, batch_size = 8, shuffle = False, num_workers=0)

In [20]:
class InterpolRegressor(pl.LightningModule):
    def __init__(self):
        super(InterpolRegressor, self).__init__()
        
        input_dim = 5
        hidden_dim_1 = 60
        hidden_dim_2 = 80
        hidden_dim_3 = 100
        hidden_dim_4 = 120
        hidden_dim_5 = 140
        hidden_dim_6 = 100
        hidden_dim_7 = 80
        hidden_dim_8 = 60
        output_dim = 1
        
        self.net = nn.Sequential(
                    nn.Linear(input_dim, hidden_dim_1),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_1, hidden_dim_2),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_2, hidden_dim_3),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_3, hidden_dim_4),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_4, hidden_dim_5),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_5, hidden_dim_6),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_6, hidden_dim_7),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_7, hidden_dim_8),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_8, output_dim)
                )

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.MSELoss()
        loss = loss(y_hat.reshape(-1), y)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        global ta
        ta = batch
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.MSELoss()
        loss = loss(y_hat.reshape(-1), y)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [21]:
data_module = InterpolDataModule()
model = InterpolRegressor()

trainer = pl.Trainer(max_epochs=3,
                     accelerator='cpu')
trainer.fit(model, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 69.5 K
------------------------------------
69.5 K    Trainable params
0         Non-trainable params
69.5 K    Total params
0.278     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [54]:
preds = model.forward(data_module.train_dataset.features)

In [55]:
mean_squared_error(preds.detach())

2.0297163